In [ ]:
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-cpu==1.7.2 fairseq gradio==3.14.0 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2
!pip3 install torch torchvision torchaudio
##!pip3 install faiss-gpu fairseq gradio==3.35.1 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2

In [ ]:
!mkdir content
!mkdir content/rmvpe-ai
%cd /kaggle/working/content/rmvpe-ai

In [ ]:
!git clone -b 'v1.1-patch-no-ui' --single-branch --depth 1 https://github.com/elcolex777/rmvpe-ai.git .

In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt -y install -qq aria2

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/3PnwhB6 -d /content/rmvpe-ai/pretrained_v2 -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/45Oa5FE -d /content/rmvpe-ai/pretrained_v2 -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/45J4v80 -d /content/rmvpe-ai/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/45XHOfP -d /content/rmvpe-ai/pretrained_v2 -o f0G40k.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/3L9mNa8 -d /content/rmvpe-ai -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/47Yxi9Y -d /content/rmvpe-ai -o rmvpe.pt

In [ ]:
DATASET = "lukhovchanka-raw"

!mkdir -p "dataset_raw"
!cp -R /kaggle/input/{DATASET}/* -t "dataset_raw/"

In [ ]:
%cd /kaggle/working/content/rmvpe-ai

In [ ]:
!mkdir -p logs/mi-test
!touch logs/mi-test/preprocess.log
#!touch logs/mi-test/filelist.txt
#!touch logs/filelist.txt

!touch logs/mi-test/extract_f0_feature.log
#python extract_f0_print.py logs/mi-test $(nproc) pm
#python extract_feature_print.py cpu 1 0 0 logs/mi-test v1


In [ ]:
# @markdown 模型名
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown 采样率
BITRATE = 40000  # @param {type:"integer"}
# @markdown 使用的进程数
THREADCOUNT = 8  # @param {type:"integer"}

!python3 trainset_preprocess_pipeline_print.py /kaggle/working/content/rmvpe-ai/dataset_raw {BITRATE} {THREADCOUNT} logs/{MODELNAME} True

In [ ]:
#!python3 extract_f0_rmvpe.py 2 0 0 "/kaggle/working/content/rmvpe-ai/logs/mi-test" True
#!python3 extract_f0_rmvpe.py 2 1 0 "/kaggle/working/content/rmvpe-ai/logs/mi-test" True

In [ ]:
# @markdown 模型名
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown 使用的进程数
THREADCOUNT = 8  # @param {type:"integer"}
# @markdown 音高提取算法
ALGO = "rmvpe"  # @param {type:"string"} pm|harvest|crepe|rmvpe

!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}

!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME} v2

In [ ]:
# @markdown 模型名
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown 采样率
MODELSAMPLE = "40k"  # @param {type:"string"}

!python3 create_filelist_print.py {MODELNAME} v2 True {MODELSAMPLE} 0

In [ ]:
# @markdown 模型名
MODELNAME = "mi-test"  # @param {type:"string"}

!python3 train_index_print.py {MODELNAME} v2

In [ ]:
# @markdown 模型名
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown 使用的GPU
USEGPU = "1"  # @param {type:"string"}
# @markdown 批大小
BATCHSIZE = 7  # @param {type:"integer"}
# @markdown 停止的epoch
MODELEPOCH = 800  # @param {type:"integer"}
# @markdown 保存epoch间隔
EPOCHSAVE = 100  # @param {type:"integer"}
# @markdown 采样率
MODELSAMPLE = "40k"  # @param {type:"string"}
# @markdown 是否缓存训练集
CACHEDATA = 0  # @param {type:"integer"}
# @markdown 是否仅保存最新的ckpt文件
ONLYLATEST = 0  # @param {type:"integer"}


!python3 train_nsf_sim_cache_sid_load_pretrain.py -e {MODELNAME} -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained_v2/f0G{MODELSAMPLE}.pth -pd pretrained_v2/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA} -sw 1 -v v2
#!python3 train_nsf_sim_cache_sid_load_pretrain.py -e {MODELNAME} -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg /kaggle/working/content/rmvpe-ai/G_55600.pth -pd /kaggle/working/content/rmvpe-ai/D_55600.pth -l {ONLYLATEST} -c {CACHEDATA} -sw 1 -v v2